# GLENS - LOGISTIC REGRESSION

Project with Jim and Lantao.

Control Simuations
* 21 simulations 2010-2030
* 3 of those continue to 2097 [1,2,3]

GLENS Simulations
* 20 simulations of 2020-2099

In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import random
import xarray as xr
import scipy.stats as stats
import random

import time
import os.path
from os import path
import subprocess
import copy as copy
import pickle

# https://www.pyimagesearch.com/2019/10/21/keras-vs-tf-keras-whats-the-difference-in-tensorflow-2-0/
import sklearn
from sklearn import metrics
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import regularizers
from tensorflow.keras import metrics
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
# import innvestigate

import seaborn as sns
import palettable

import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy as ct
import cartopy.crs as ccrs
import cmocean as cmocean
from mpl_toolkits.axes_grid1 import make_axes_locatable
import palettable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage import gaussian_filter

import experiment_settings

# Plotting
mpl.rcParams['figure.dpi']= 150
dpiFig = 300.
CL = 0.
mapProj = ct.crs.EqualEarth(central_longitude = CL)

# random numbers
%env PYTHONHASHSEED=99
np.random.seed(99)
tf.random.set_seed(99)

env: PYTHONHASHSEED=99


In [2]:
print(f"numpy version = {np.__version__}")
print(f"xarray version = {xr.__version__}")  
print(f"tensorflow version = {tf.__version__}")  

numpy version = 1.22.2
xarray version = 2022.3.0
tensorflow version = 2.7.0


In [3]:
EXP_NAME = 'exp2a' #'exp0_tx90p' #'exp1_r95ptot' 'exp2_t'
OVERWRITE_MODEL = True
SAVE_DATA = False

#-------------------------------------------------------
settings = experiment_settings.get_settings(EXP_NAME)
display(settings)

fileDir = 'data/GLENS/'

{'var': 'T',
 'n_train_val_test': (16, 4, 0),
 'ens_seed': (3529, 4529, 5529, 6529, 7529),
 'net_seed': (2222, 3333, 4444, 5555, 6666),
 'land_only': True,
 'remove_mean': False,
 'network_type': 'logistic',
 'hiddens': [1],
 'dropout_rate': 0.0,
 'ridge_param': 0.0,
 'learning_rate': 0.001,
 'n_epochs': 75,
 'batch_size': 32,
 'patience': 50,
 'lr_epoch_bound': 150,
 'exp_name': 'exp2a'}

In [4]:
if(settings["var"]=='TX90p' or settings["var"]=='TX10p' or settings["var"]=='R95pTOT'):
    extremeBool = True    
else:
    extremeBool = False

## ML Network

In [5]:
lr_epoch_bound = settings["lr_epoch_bound"]
def scheduler(epoch, lr):
    if epoch < lr_epoch_bound:
        return lr
    else:
        return lr #* tf.math.exp(-0.05)
lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler,verbose=0)


## Metrics

In [6]:
def get_predictions(X, labels, features):
    y_pred = np.asarray(np.round(model.predict(X)),dtype='float')
    y_pred_like = model.predict(X)
#     y_pred_like = np.fromiter((row[index] for row, index in zip(y_pred_like, np.asarray(labels.flatten(),dtype='int'))), dtype='float')
    
    y_pred_like = y_pred_like.reshape((features.shape[0],features.shape[1]))
    y_pred = y_pred.reshape((features.shape[0],features.shape[1]))
    acc = np.asarray(np.asarray(np.equal(y_pred,labels),dtype='int32'),dtype='float') # convert True/False to zeros and ones
    
    # replace nans with nan
    jnan = np.isnan(features[:,:,0,0])
    y_pred_like[jnan] = np.nan
    y_pred[jnan] = np.nan
    acc[jnan] = np.nan

    return y_pred_like, y_pred, acc

## Load the data

In [7]:
df_mask = xr.open_dataset('data/GLENS/landSeaMask.r90x45.nc')
# print(df_mask)
landSeaMask = df_mask['TN10p'].values[0,0,:,:]
lats = df_mask['lat']
lons = df_mask['lon']
df_mask.close()

landSeaMask[np.isnan(landSeaMask)==False] = 1.
landSeaMask[np.isnan(landSeaMask)==True] = np.nan
landSeaMask = landSeaMask.reshape(len(lats)*len(lons),)
landSeaMask

array([ 1.,  1.,  1., ..., nan, nan, nan], dtype=float32)

In [8]:
if(extremeBool == False):
    middletext = '.r90x45.shift.annual'

#     print('** control has different months for the same year **')    
    runType = 'control'
    dsc = xr.open_mfdataset(fileDir + runType + '*.' + settings["var"] + middletext + '.nc', concat_dim='ensmember', combine='nested')    
    time_dsc = dsc['time.year']
    
    runType = 'feedback'
    dsf = xr.open_mfdataset(fileDir + runType + '*.' + settings["var"] + middletext + '.nc', concat_dim='ensmember', combine='nested')
    time_dsf = dsf['time.year']
    
    if(settings["var"]=='T'):
        dsf = dsf.sel(lev=1000.)
        dsc = dsc.sel(lev=1000.)    
    
else:    # get the extremes data
    varFile = 'TREFHTMX'
    if(settings["var"]=='R95pTOT'):
        varFile = 'PRECT'

    middletext = '.swap.r90x45'
    runType = 'control'
    filename = fileDir + 'b.e15.B5505C5WCCML45BGCR.f09_g16.' + runType + '.extreme.cam.h3.' + varFile + '.20100101-20971231' + middletext + '.nc'
    dsc = xr.open_dataset(filename)
    dsc = dsc.transpose('ensemble', 'time', 'lat', 'lon')
    time_dsc = np.arange(2010, 2098, 1)
    
    runType = 'feedback'
    filename = fileDir + 'b.e15.B5505C5WCCML45BGCR.f09_g16.' + runType + '.extreme.cam.h3.' + varFile + '.20200101-20991231' + middletext + '.nc'
    dsf = xr.open_dataset(filename)
    dsf = dsf.transpose('ensemble', 'time', 'lat', 'lon')
    time_dsf = np.arange(2020,2100,1)
#------------------------------------------------------------------------
# CONTROL
itime = np.where((time_dsc>=2021) & (time_dsc<=2097))[0]    
featuresC = dsc[settings["var"]][:,itime,:,:].values.astype('float')
time_dsc = time_dsc[itime]

if(featuresC.shape[0]==21):
    featuresC = featuresC[:-1,:,:,:]
elif(featuresC.shape[0]==3):
    filename = fileDir + 'b.e15.B5505C5WCCML45BGCR.f09_g16.' + 'control' + '.extreme.cam.h3.' + varFile + '.20210101-20301231' + middletext + '.nc'
    dsc20 = xr.open_dataset(filename)
    dsc20 = dsc20.transpose('ensemble', 'time', 'lat', 'lon')
    featuresC20 = dsc20[settings["var"]].values.astype('float')
    hold = np.nan*np.empty((20,77,45,90))
    hold[:3,:,:,:] = featuresC
    hold[3:,:featuresC20.shape[1],:,:] = featuresC20[3:,:,:,:]
    featuresC = copy.deepcopy(hold)
#------------------------------------------------------------------------    
# FEEDBACK
itime = np.where((time_dsf>=2021) & (time_dsf<=2097))[0]   
featuresF = dsf[settings["var"]][:,itime,:,:].values.astype('float')
time_dsf = time_dsf[itime]
if(featuresF.shape[0]==21):
    featuresF = featuresF[:-1,:,:,:]

featuresC_raw = copy.deepcopy(featuresC)
featuresF_raw = copy.deepcopy(featuresF)
print(featuresC_raw.shape)
print(featuresF_raw.shape)    

(20, 77, 45, 90)
(20, 77, 45, 90)


# Train the model

In [9]:
exp_dict = {}

batch_size = settings["batch_size"]
verbose = False
ridgeVal = settings["ridge_param"]
hiddens = settings["hiddens"]
landOnly = settings["land_only"]
niter = settings["n_epochs"]
lr_here = settings["learning_rate"]
rmMean = settings["remove_mean"]

for model_num in np.arange(0,len(settings["ens_seed"])):
    net_seed = settings["net_seed"][model_num] 
    ens_seed = settings["ens_seed"][model_num] 
    print(ens_seed)
    
    #=====================================================================================
    # get the data
    rng = np.random.default_rng(ens_seed)    
    valNum = settings["n_train_val_test"][1]
    val_member = rng.choice([0,1,2],1, replace=False)
    train_members = rng.choice(np.arange(3,np.shape(featuresC_raw)[0]),np.shape(featuresC_raw)[0]-3, replace=False)
    train_members = np.append(np.setdiff1d([0,1,2],[val_member]),train_members)
    number_list = np.append(train_members,val_member)   

    #-------------------
    print('ensemble list = ' + str(number_list) + '\n')
    featuresC = featuresC_raw[number_list,:,:,:]
    featuresF = featuresF_raw[number_list,:,:,:]

    #-------------------
    # train/test split
    print('training members = ' + str(number_list[:-valNum]))
    print('validate members = ' + str(number_list[-valNum:]))
    featuresF_train = featuresF[:-valNum]
    featuresC_train = featuresC[:-valNum]
    featuresF_test = featuresF[-valNum:]
    featuresC_test = featuresC[-valNum:]

    labelsC_train = np.ones((featuresC_train.shape[0],featuresC_train.shape[1]))*0
    labelsF_train = np.ones((featuresF_train.shape[0],featuresF_train.shape[1]))*1
    labelsC_test = np.ones((featuresC_test.shape[0],featuresC_test.shape[1]))*0
    labelsF_test = np.ones((featuresF_test.shape[0],featuresF_test.shape[1]))*1

    #-------------------
    # concatenate
    features_train = np.append(featuresC_train,featuresF_train,axis=0)
    features_test = np.append(featuresC_test,featuresF_test,axis=0)
    labels_train = np.append(labelsC_train,labelsF_train,axis=0)
    labels_test = np.append(labelsC_test,labelsF_test,axis=0)

    #-------------------
    # standardize
    featuresMean = np.nanmean(features_train,axis=(0,1))
    featuresStd = np.nanstd(features_train,axis=(0,1))

    features_train = (features_train-featuresMean)/featuresStd
    features_test = (features_test-featuresMean)/featuresStd
    
    #=====================================================================================    

    #=========================================================================
    #    FINALIZE THE DATA
    #=========================================================================

    # convert labels to categorical
    y_train = tf.keras.utils.to_categorical(labels_train)
    y_test = tf.keras.utils.to_categorical(labels_test)

    # grab 2021-2030 for training
    itime_train = np.where(time_dsc<=2097)[0]
    X_train = features_train[:,itime_train,:,:]
    X_test = features_test[:,itime_train,:,:]
    y_train = y_train[:,itime_train,:]
    y_test = y_test[:,itime_train,:]

    # flatten the training and testing 
    X_train = X_train.reshape(X_train.shape[0]*X_train.shape[1],X_train.shape[2]*X_train.shape[3])
    X_test = X_test.reshape(X_test.shape[0]*X_test.shape[1],X_test.shape[2]*X_test.shape[3])
    y_train = y_train.reshape((y_train.shape[0]*y_train.shape[1],y_train.shape[2]))
    y_test = y_test.reshape((y_test.shape[0]*y_test.shape[1],y_test.shape[2]))

    if(landOnly==True):
        X_train = X_train*landSeaMask
        X_test = X_test*landSeaMask
        landText = '_landOnly'
    else:
        landText = ''
        X_train = X_train
        X_test = X_test

    if(rmMean==True):
        print('removing the mean...')
        X_train = X_train - np.nanmean(X_train,axis=1)[:,np.newaxis]
        X_test = X_test - np.nanmean(X_test,axis=1)[:,np.newaxis]    

    # replace NaN with zeros    
    X_train[np.isnan(X_train)] = 0.
    X_test[np.isnan(X_test)] = 0.

    # make output predict whether from control
    y_train = y_train[:,1:]
    y_test = y_test[:,1:]
    #=========================================================================

    filename = settings["exp_name"] + '_ensSeed' + str(ens_seed) + '_netSeed' + str(net_seed)

    #=========================================================================
    #     TRAIN THE MODEL
    #=========================================================================
    model = Sequential()
    model.add(Dense(y_train.shape[1], input_shape=(X_train.shape[1],), activation='sigmoid', use_bias=True, kernel_regularizer=regularizers.l1_l2(l1=0.00, l2=ridgeVal),bias_initializer=tf.keras.initializers.RandomNormal(seed=net_seed),
                kernel_initializer=tf.keras.initializers.RandomNormal(seed=net_seed)))
    model.compile(optimizer=optimizers.SGD(lr=lr_here, momentum=0.9, nesterov=True),  #Adadelta .Adam()
                 loss = 'binary_crossentropy',
                 metrics=[metrics.categorical_accuracy],)
    print(model.summary())

    print('starting training...')
    history = model.fit(X_train, y_train, 
                        batch_size=batch_size, 
                        epochs=niter, 
                        shuffle=True, 
                        verbose=verbose, 
                        callbacks=[lr_callback,], 
                        validation_data=(X_test,y_test))
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    y_pred_like_train, y_pred_train, acc_train = get_predictions(X_train,labels_train,features_train)
    y_pred_like_test, y_pred_test, acc_test = get_predictions(X_test,labels_test,features_test)
    d = {"y_pred_like_train":y_pred_like_train,
         "y_pred_train": y_pred_train,
         "acc_train": acc_train,
         "y_pred_like_test": y_pred_like_test,
         "y_pred_test": y_pred_test,
         "acc_test": acc_test,
         "filename": filename,
         "exp_name": settings["exp_name"],
         "net_seed": net_seed,
         "ens_seed": ens_seed,
    }
    exp_dict[filename] = d
    
    #=========================================================================
    #     SAVE OUTPUT
    #=========================================================================
    model.save('saved_models/model_' + filename + '.h5')


with open("saved_predictions/" + settings["exp_name"] + ".pickle", 'wb') as handle:
    pickle.dump(exp_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
print('training and saving done.')


3529
ensemble list = [ 1  2 16  6 10 14 11 15 13 12  8  3  5  9  7  4 17 18 19  0]

training members = [ 1  2 16  6 10 14 11 15 13 12  8  3  5  9  7  4]
validate members = [17 18 19  0]


2022-04-08 12:14:30.417855: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/anaconda3/envs/env-noah/lib/python3.9/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 4051      
                                                                 
Total params: 4,051
Trainable params: 4,051
Non-trainable params: 0
_________________________________________________________________
None
starting training...
4529
ensemble list = [ 0  1 13 15  8  9 17 19 14 12 16 10  7  4  6 18 11  5  3  2]

training members = [ 0  1 13 15  8  9 17 19 14 12 16 10  7  4  6 18]
validate members = [11  5  3  2]
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 1)                 4051      
                                                                 
Total params: 4,051
Trainable params: 4,051
Non-trainable params: 0
________________